Evalvuate the eror 
The objective of the eval pred is to first figure out some info about the evaluation metric. The evaluation metric Consists of two parts. Ftist is a rank based one. We rank the customers that have defulted based on the predicted probability. choose the highest .04(#1 + #0*20). 

This is equvalent to the recall level of 4% the doc states. That is, of all the customers that are going to deafult, we aim to recall at least 96% of them!! 


# Q:
 - What does a recall of 4% mean? How do we train for a recall of a 4%? 

In [46]:
import torch 
import pandas as pd
from pd.params import *

In [47]:
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

from pd.nn.model import Conv
from pd.data.loader import CustomerData, DTwithLabelRatio
from pd.nn.train_utils import train_torch_model
from pd.metric import amex_metric
from pd.params import *
from pd.pred import pred_test_npy as predict

In [48]:
def amex_metric(y_true, y_pred, return_components=False):
    """Amex metric for ndarrays"""
    def top_four_percent_captured(df) -> float:
        """Corresponds to the recall for a threshold of 4 %"""
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()
        
    def weighted_gini(df) -> float:
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
        total_pos = (df['target'] * df['weight']).sum()
        df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
        df['lorentz'] = df['cum_pos_found'] / total_pos
        df['gini'] = (df['lorentz'] - df['random']) * df['weight']
        return df['gini'].sum()

    def normalized_weighted_gini(df) -> float:
        """Corresponds to 2 * AUC - 1"""
        df2 = pd.DataFrame({'target': df.target, 'prediction': df.target})
        df2.sort_values('prediction', ascending=False, inplace=True)
        return weighted_gini(df) / weighted_gini(df2)

    df = pd.DataFrame({'target': y_true.ravel(), 'prediction': y_pred.ravel()})
    df.sort_values('prediction', ascending=False, inplace=True)
    g = normalized_weighted_gini(df)
    d = top_four_percent_captured(df)

    if return_components: return g, d, 0.5 * (g + d)
    return 0.5 * (g + d)

In [7]:
def top_four_percent_captured(df) -> float:
        """Corresponds to the recall for a threshold of 4 %"""
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()
    

In [6]:
import numpy as np
import torch 
from pd.nn.model import Conv

In [18]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
from bes.nn.es_module import ESModule


In [10]:
train_data = np.load(OUTDIR+"train13_raw_all_data.npy")
train_labels = np.load(OUTDIR+"train13_raw_all_labels.npy")

In [9]:

from torch.utils.data import Dataset, dataloader

test_data = np.load(OUTDIR+"train_raw_all_data.npy", mmap_mode="r+")
train_labels = np.load(OUTDIR+"train_raw_all_labels.npy")
class Data(Dataset):
    def __init__(self, data):
        self.data = data
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):        
        return torch.from_numpy(self.data[index]), index

train_dataset = Data(test_data, )
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE)


In [19]:
model_name = "conv13_32_all"
model = Conv(conv_channels=32)
model_param = torch.load(OUTDIR+model_name)
model.load_state_dict(model_param)

<All keys matched successfully>

In [14]:
X_train, X_test, y_train, y_test = train_test_split(train_data, train_labels, test_size=1/9, random_state=1, shuffle=True)
validation_data = (X_test, y_test)


In [16]:
X_test, y_test = validation_data
val_features = torch.from_numpy(X_test)
val_pred = model(val_features)
val_metrix, val_gini, val_recall = amex_metric(y_test, val_pred.detach().numpy(), return_components=True)


In [17]:
val_metrix, val_gini, val_recall

(0.8106466362523513, 0.9281171813848559, 0.6931760911198468)

In [13]:
val_metrix, val_gini, val_recall

(0.8076595984542532, 0.9279778102472538, 0.6873413866612527)

In [9]:
preds = np.load(OUTDIR+"c13_preds.npy")

In [ ]:
a, b, c = amex_metric(train_labels, preds, return_components=True)

In [13]:
preds = np.zeros(len(train_dataset))
model.eval()
for idx, (feat, indices) in enumerate(train_loader):
        batch_pred = model(feat)
        preds[indices.numpy()] = batch_pred.detach().numpy().reshape(-1, )
        #preds.append(batch_pred)
        print(amex_metric(train_labels[indices.numpy()], preds[indices.numpy()], return_components=True))

(0.9189522846823319, 0.6626991745056633, 0.7908257295939976)
(0.9117926612179975, 0.6322059953881629, 0.7719993283030802)
(0.9137001253827733, 0.630639278172394, 0.7721697017775837)
(0.910439040794739, 0.6391986391986392, 0.7748188399966891)
(0.9143557163604561, 0.6452611218568666, 0.7798084191086614)
(0.9117057372875503, 0.6510107015457788, 0.7813582194166646)
(0.907421621330389, 0.6261520737327189, 0.7667868475315539)
(0.9117512332341383, 0.6370797310278579, 0.7744154821309981)
(0.9146816478243461, 0.6332438650306749, 0.7739627564275104)
(0.911634259072624, 0.6298299845440495, 0.7707321218083367)
(0.9122821673600039, 0.6259376233714963, 0.76910989536575)
(0.9091516211022168, 0.630766223612197, 0.769958922357207)
(0.9177579459881187, 0.6546998648909477, 0.7862289054395332)
(0.9128007189670488, 0.6530612244897959, 0.7829309717284223)
(0.9095910951030811, 0.6390001908032819, 0.7742956429531815)
(0.9122655531659488, 0.6271450858034321, 0.7697053194846905)
(0.9123261570727688, 0.645030032

In [18]:
amex_metric(train_labels[indices.numpy()], preds[indices.numpy()], return_components=True)


(0.47234747244543596, 0.09336767546683837, 0.28285757395613714)

In [25]:
%load_ext memory_profiler


In [10]:
X_train, X_test, y_train, y_test = train_test_split(train_data, train_labels, test_size=1/9, random_state=0, shuffle=True)
validation_data = (X_test, y_test)

train_dataset = CustomerData(X_train, train_labels=y_train)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE)


In [14]:
model_name = "conv_all_13"

model = Conv(input_dim=X_train.shape[-1], conv_channels=25)
model = train_torch_model(model, train_loader, num_epochs=50, validation_data=validation_data, 
                        output_model_name=model_name)
 

0, BCE loss: 0.746, amex train: 0.033, val 0.000
0, BCE loss: 0.880, amex train: 0.146, val 0.000
0, BCE loss: 0.725, amex train: 0.220, val 0.000
0, BCE loss: 0.568, amex train: 0.288, val 0.000
0, BCE loss: 0.526, amex train: 0.314, val 0.000
0, BCE loss: 0.582, amex train: 0.345, val 0.000
0, BCE loss: 0.607, amex train: 0.355, val 0.000
0, BCE loss: 0.571, amex train: 0.358, val 0.000
0, BCE loss: 0.529, amex train: 0.384, val 0.000
0, BCE loss: 0.511, amex train: 0.386, val 0.000
0, BCE loss: 0.519, amex train: 0.391, val 0.000
0, BCE loss: 0.531, amex train: 0.378, val 0.000
0, BCE loss: 0.546, amex train: 0.387, val 0.000
0, BCE loss: 0.537, amex train: 0.388, val 0.000
0, BCE loss: 0.542, amex train: 0.388, val 0.000
0, BCE loss: 0.529, amex train: 0.396, val 0.000
0, BCE loss: 0.520, amex train: 0.399, val 0.000
0, BCE loss: 0.494, amex train: 0.387, val 0.000
1, BCE loss: 0.507, amex train: 0.408, val 0.000
1, BCE loss: 0.503, amex train: 0.387, val 0.000
1, BCE loss: 0.511, 

: 

: 

(343141, 13, 188)

# D analysis of GBM

In [5]:
import numpy as np
import pandas as pd
from pd.params import *
def get_agg_data(data_dir="train_agg_mean_q5_q95_q5_q95.npz"):
    d = np.load(OUTDIR+data_dir)
    #train_data = np.concatenate((d["d2"].astype(np.int32), d["d1"].reshape(d["d1"].shape[0], -1)), axis=1)
    train_labels = d["labels"]
    df2 = pd.DataFrame(d["d2"].astype(np.int32))
    df = pd.DataFrame(d["d1"].reshape(d["d1"].shape[0], -1))
    df = pd.concat((df2, df), axis=1,)
    df.columns = [f"c{i}" for i in range(df.shape[1])]
    cat_indices = list(np.arange(d["d2"].shape[1]))

    return df, train_labels, cat_indices

In [57]:
df, train_labels, cat_indices = get_agg_data(data_dir="train_agg_mean_q5_q95_q5_q95.npz")        

In [58]:
from sklearn.model_selection import StratifiedKFold, train_test_split
X_train, X_test, y_train, y_test = train_test_split(df, train_labels, test_size=1/9, random_state=0, shuffle=True)
validation_data = (X_test, y_test)

In [59]:
import lightgbm as lgb 
model = lgb.Booster(model_file=OUTDIR+"lgbm_agg")


In [60]:
y_pred = model.predict(X_test)

In [61]:
amex_metric(y_test, y_pred, return_components=True)

(0.9247766447649399, 0.6687252211039951, 0.7967509329344675)

In [64]:
df = pd.DataFrame({"pred":y_pred, "target":y_test.reshape(-1)})

In [66]:
df.sort_values(by="pred", ascending=False, inplace=True)
df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
df['weight_cumsum'] = df['weight'].cumsum()
#df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        

In [16]:
four_pct_cutoff = int(0.04 * df['weight'].sum())


In [34]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)



In [18]:
df.sort_values(by=["pred", "weight"], ascending=False).reset_index(drop=True).to_csv(OUTDIR+"pred.csv")

In [36]:
df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]

In [40]:
df_cutoff[["target", "c0"]].value_counts()

target  c0
1.0     13    7440
0.0     13     932
1.0     12     189
        10     141
        8      126
        9      110
        2      109
        7      105
        11     102
        6      101
        3      101
        5       94
        1       82
        4       71
0.0     9       25
        12      22
        1       18
        4       17
        3       17
        2       14
        10      14
        7       10
        6       10
        11       9
        8        8
        5        6
dtype: int64

In [24]:
a = df.loc[df['weight_cumsum'] >= four_pct_cutoff]
a[a.target == 1].shape

(4345, 4)

In [42]:
a = df_cutoff[df_cutoff.c0==13]

In [ ]:
import plotly.express as px
fig = px.line(a, x="feature", y=['gbm_recall', 'gbm_gini'], hue="target")
fig.show()

# Error Analysis of the C13 

In [49]:
train_data = np.load(OUTDIR+"train13_raw_all_data.npy")
train_labels = np.load(OUTDIR+"train13_raw_all_labels.npy")

In [50]:
X_train, X_test, y_train, y_test = train_test_split(train_data, train_labels, test_size=1/9, random_state=0, shuffle=True)
validation_data = (X_test, y_test)

In [53]:
import joblib
import lightgbm as lgb 

model = joblib.load(OUTDIR+f'lgbm13.pkl')

In [56]:
y_pred = model.predict(X_test.reshape(X_test.shape[0], -1))
amex_metric(y_test, y_pred, return_components=True)

(0.9323274209347154, 0.6966805400466958, 0.8145039804907056)

In [67]:
y_pred.shape 

(50991,)

In [68]:
import json 
with open(OUTDIR+'train_agg1_mean_q5_q95_q5_q95_id.json', 'r') as f:
            train_id_dict = json.load(f)

In [83]:
train_customers_count = pd.read_parquet(TRAINDATA).customer_ID.value_counts().to_dict()


In [ ]:
train_id_dict1 = {val:key for key, val in train_id_dict.items()}

In [84]:
# get c13 data 

c13_indices = [int(i) for c, i in train_id_dict1.items() if train_customers_count[c]==13]

[0,
 1,
 2,
 3,
 4,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 33,
 35,
 36,
 37,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 56,
 57,
 58,
 60,
 61,
 62,
 63,
 66,
 67,
 68,
 69,
 71,
 72,
 73,
 74,
 75,
 77,
 78,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 96,
 97,
 98,
 99,
 100,
 101,
 103,
 105,
 106,
 107,
 108,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 152,
 153,
 154,
 156,
 157,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 168,
 169,
 170,
 171,
 173,
 174,
 175,
 176,
 177,
 179,
 180,
 181,
 182,
 184,
 185,
 186,
 187,
 188,
 189,
 190,
 193,
 196,
 197,
 198,
 199,
 200,
 201,
 202,
 203,
 204,
 206,
 207,
 208,
 209,
 210,
 211,
 212,
 213,
 215,
 216,
 217,
 